## **Telco Customer Churn Prediction Model**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

from imblearn.over_sampling import SMOTE
from flaml import AutoML


def load_data():
    df = pd.read_csv(r"C:\MachineLearningModel_Deployment\data\churn.csv")
    print("Data Loaded Successfully")
    return df


def split_features_target(df):
    X = df.drop("Churn", axis=1)

    # Encode target properly for FLAML
    y = df["Churn"].map({"No": 0, "Yes": 1})

    print("Feature and Target Split Done")
    print("Target Classes:", y.unique())
    return X, y


def split_train_test(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )

    print("Train Test Split Completed")
    return X_train, X_test, y_train, y_test


def create_preprocessor(X):
    num_cols = X.select_dtypes(include=["int64", "float64"]).columns
    cat_cols = X.select_dtypes(include=["object"]).columns

    numeric_pipeline = SimpleImputer(strategy="median")

    categorical_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer([
        ("num", numeric_pipeline, num_cols),
        ("cat", categorical_pipeline, cat_cols)
    ])

    print("Preprocessor Created")
    return preprocessor


def apply_preprocessing(preprocessor, X_train, X_test):
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    print("Preprocessing Applied")
    return X_train_processed, X_test_processed


def apply_smote(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    print("SMOTE Applied")
    print("Class Distribution After SMOTE:", np.bincount(y_resampled))

    return X_resampled, y_resampled


def train_flaml(X_train, y_train):
    automl = AutoML()

    settings = {
        "time_budget": 60,
        "metric": "accuracy",
        "task": "classification",
        "log_file_name": "flaml.log",
        "seed": 42
    }

    automl.fit(X_train=X_train, y_train=y_train, **settings)

    print("FLAML Training Completed")
    print("Best Model:", automl.model.estimator)

    return automl


def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    print("\nAccuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n")
    print(classification_report(y_test, y_pred))


def main():
    df = load_data()

    X, y = split_features_target(df)

    X_train, X_test, y_train, y_test = split_train_test(X, y)

    preprocessor = create_preprocessor(X)

    X_train_processed, X_test_processed = apply_preprocessing(
        preprocessor, X_train, X_test
    )

    X_resampled, y_resampled = apply_smote(X_train_processed, y_train)

    model = train_flaml(X_resampled, y_resampled)

    evaluate_model(model, X_test_processed, y_test)


if __name__ == "__main__":
    main()

Data Loaded Successfully
Feature and Target Split Done
Target Classes: [0 1]
Train Test Split Completed
Preprocessor Created
Preprocessing Applied
SMOTE Applied
Class Distribution After SMOTE: [4139 4139]
[flaml.automl.logger: 02-16 21:19:01] {2375} INFO - task = classification
[flaml.automl.logger: 02-16 21:19:01] {2386} INFO - Evaluation method: holdout
[flaml.automl.logger: 02-16 21:19:01] {2489} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 02-16 21:19:01] {2606} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 02-16 21:19:01] {2911} INFO - iteration 0, current learner lgbm


C:\Users\USER\AppData\Local\Temp\ipykernel_6368\3270120721.py:47: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = X.select_dtypes(include=["object"]).columns


[flaml.automl.logger: 02-16 21:19:01] {3046} INFO - Estimated sufficient time budget=926s. Estimated necessary time budget=21s.
[flaml.automl.logger: 02-16 21:19:01] {3097} INFO -  at 0.1s,	estimator lgbm's best error=2.0894e-01,	best estimator lgbm's best error=2.0894e-01
[flaml.automl.logger: 02-16 21:19:01] {2911} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 02-16 21:19:01] {3097} INFO -  at 0.2s,	estimator lgbm's best error=2.0894e-01,	best estimator lgbm's best error=2.0894e-01
[flaml.automl.logger: 02-16 21:19:01] {2911} INFO - iteration 2, current learner lgbm
[flaml.automl.logger: 02-16 21:19:02] {3097} INFO -  at 0.3s,	estimator lgbm's best error=1.9082e-01,	best estimator lgbm's best error=1.9082e-01
[flaml.automl.logger: 02-16 21:19:02] {2911} INFO - iteration 3, current learner sgd


c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[flaml.automl.logger: 02-16 21:19:02] {3097} INFO -  at 0.6s,	estimator sgd's best error=2.9106e-01,	best estimator lgbm's best error=1.9082e-01
[flaml.automl.logger: 02-16 21:19:02] {2911} INFO - iteration 4, current learner xgboost
[flaml.automl.logger: 02-16 21:19:02] {3097} INFO -  at 0.7s,	estimator xgboost's best error=2.3551e-01,	best estimator lgbm's best error=1.9082e-01
[flaml.automl.logger: 02-16 21:19:02] {2911} INFO - iteration 5, current learner lgbm
[flaml.automl.logger: 02-16 21:19:02] {3097} INFO -  at 0.8s,	estimator lgbm's best error=1.7029e-01,	best estimator lgbm's best error=1.7029e-01
[flaml.automl.logger: 02-16 21:19:02] {2911} INFO - iteration 6, current learner lgbm
[flaml.automl.logger: 02-16 21:19:02] {3097} INFO -  at 0.9s,	estimator lgbm's best error=1.7029e-01,	best estimator lgbm's best error=1.7029e-01
[flaml.automl.logger: 02-16 21:19:02] {2911} INFO - iteration 7, current learner lgbm


...

[flaml.automl.logger: 02-16 21:19:50] {3097} INFO -  at 48.6s,	estimator sgd's best error=1.4734e-01,	best estimator xgboost's best error=1.4010e-01
[flaml.automl.logger: 02-16 21:19:50] {2911} INFO - iteration 200, current learner sgd
[flaml.automl.logger: 02-16 21:19:50] {3097} INFO -  at 49.1s,	estimator sgd's best error=1.4734e-01,	best estimator xgboost's best error=1.4010e-01
[flaml.automl.logger: 02-16 21:19:50] {2911} INFO - iteration 201, current learner sgd
[flaml.automl.logger: 02-16 21:19:51] {3097} INFO -  at 49.5s,	estimator sgd's best error=1.4734e-01,	best estimator xgboost's best error=1.4010e-01
[flaml.automl.logger: 02-16 21:19:51] {2911} INFO - iteration 202, current learner lgbm
[flaml.automl.logger: 02-16 21:19:51] {3097} INFO -  at 49.7s,	estimator lgbm's best error=1.4251e-01,	best estimator xgboost's best error=1.4010e-01
[flaml.automl.logger: 02-16 21:19:51] {2911} INFO - iteration 203, current learner xgboost


c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[flaml.automl.logger: 02-16 21:19:51] {3097} INFO -  at 50.2s,	estimator xgboost's best error=1.3647e-01,	best estimator xgboost's best error=1.3647e-01
[flaml.automl.logger: 02-16 21:19:51] {2911} INFO - iteration 204, current learner sgd
[flaml.automl.logger: 02-16 21:19:52] {3097} INFO -  at 50.3s,	estimator sgd's best error=1.4734e-01,	best estimator xgboost's best error=1.3647e-01
[flaml.automl.logger: 02-16 21:19:52] {2911} INFO - iteration 205, current learner sgd
[flaml.automl.logger: 02-16 21:19:52] {3097} INFO -  at 50.6s,	estimator sgd's best error=1.4734e-01,	best estimator xgboost's best error=1.3647e-01
[flaml.automl.logger: 02-16 21:19:52] {2911} INFO - iteration 206, current learner sgd
[flaml.automl.logger: 02-16 21:19:52] {3097} INFO -  at 51.0s,	estimator sgd's best error=1.4734e-01,	best estimator xgboost's best error=1.3647e-01
[flaml.automl.logger: 02-16 21:19:52] {2911} INFO - iteration 207, current learner xgb_limitdepth
[flaml.automl.logger: 02-16 21:19:53] {30

c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(
c:\MachineLearningModel_Deployment\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


[flaml.automl.logger: 02-16 21:20:04] {3097} INFO -  at 62.7s,	estimator lrl1's best error=2.4155e-01,	best estimator xgboost's best error=1.3647e-01
[flaml.automl.logger: 02-16 21:20:04] {3359} INFO - retrain xgboost for 0.5s
[flaml.automl.logger: 02-16 21:20:04] {3362} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=1.0, colsample_bynode=None,
              colsample_bytree=np.float64(0.8865773413664049), device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints=None,
              learning_rate=np.float64(0.13447115941821117), max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=12,
              min_child_weight=np.float64(1.7322532388076908), missing=n